# Subspace Digital Twin, Initial Conditions Run

*Shawn Anderson, January 2024*

In this notebook, we examine medianl behavior over the first 90 days.

## Part 1. Dependences & Set-up

Autoreload modules while developing.

In [70]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import pandas as pd
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

import hvplot.pandas
hvplot.extension('bokeh')

from bokeh.models import HoverTool
import holoviews as hv

from bokeh.palettes import Turbo256, Category20

from subspace_model.util import g

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Part 2. Load Simulation Data

Load the simulation results data.

In [71]:
sim_df = pd.read_pickle(
    "../data/simulations/reference_subsidy_sweep-2024-01-23_21-36-38.pkl.gz"
).drop(['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'], axis=1)

In [72]:
sim_df.head(5)

,days_passed,blocks_passed,circulating_supply,user_supply,earned_supply,issued_supply,earned_minus_burned_supply,total_supply,sum_of_stocks,block_utilization,dsf_relative_disbursal_per_day,reward_issuance_balance,other_issuance_balance,operators_balance,nominators_balance,holders_balance,farmers_balance,staking_pool_balance,fund_balance,burnt_balance,nominator_pool_shares,operator_pool_shares,block_reward,history_size,space_pledged,allocated_tokens,buffer_size,reference_subsidy,average_base_fee,average_priority_fee,average_compute_weight_per_tx,average_transaction_size,transaction_count,average_compute_weight_per_bundle,average_bundle_size,bundle_count,compute_fee_volume,storage_fee_volume,rewards_to_nominators,run,average_compute_weight_per_budle,label,environmental_label,max_credit_supply
0,0,0.0,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,NaN,0,0,0.0,0.0,0.00000,0.0,0.0,0.0,256,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,1,NaN,constant-single-component,standard,3000000000
14,1,14400.0,13.698630,13.698630,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.684932,13.013699,0.0,0.000000,0.0,0.0,0.0,13.698630,2038498852864,1572944000000,0.0,60045568.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.000000,0.000000,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
28,2,28800.0,27.363014,27.397260,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,1.368151,25.994863,0.0,0.034247,0.0,0.0,0.0,13.698630,4076997705728,3145888000000,0.0,120091136.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.342466,0.342466,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
42,3,43200.0,41.201036,41.303690,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,2.060052,39.140985,0.0,0.102654,0.0,0.0,0.0,13.906430,6115764994048,4718832000000,0.0,45918976.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.684075,0.684075,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
56,4,57600.0,55.211744,55.417401,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000745,0.000745,2.829882,52.380372,0.0,0.205657,0.0,0.0,0.0,14.113711,8154263846912,6291776000000,0.0,105964544.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.955515,1.030026,0.0,1,1.000000e+10,constant-single-component,standard,3000000000


Simulation Runs.

In [73]:
sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='Days').head()

,run,label,environmental_label,Days
0,1,constant-single-component,standard,732
1,2,hybrid-single-component,standard,732
2,3,hybrid-two-components,standard,732


### Coloring Metrics
Use a constant mapping from columns to colors

In [74]:
color_palette = Category20
# columns_to_color = sorted(list(set(sim_df.columns) - {'environmental_label', 'label', 'run', 'blocks_passed', 'days_passed'}))
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}


sim_df.count().to_frame().T.hvplot.bar(y=columns_to_color, color=[column_colors[c] for c in columns_to_color], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

:Bars   [index,Variable]   (value)

In [75]:
def snake_to_title(s):
    """Utility function used for printing chart titles and labels as Title Case.
    Example:
    snake_to_caps('snake_case')
    >>> 'Snake Case'
    """
    
    return ' '.join(word.capitalize() for word in s.split('_'))

def fan_chart_quantile_median(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of median for a series."""

    # min, max, median
    fan_df = df.groupby('days_passed')[column].agg(['min', 'max', 'median'])

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Fan Chart', ylabel=f'{column}_min_max_median')

    # Median curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    median_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Median', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return median_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='min', y2='max', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * median_chart
    return chart


def fan_chart_quantile(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of quantile for a series."""

    # 25%, 50%, 75%
    fan_df = df.groupby('days_passed')[column].quantile([0.25, 0.5, 0.75]).unstack().rename(columns={0.50:'median', 0.25:'0.25',0.75:'0.75'})

    # return fan_df

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Quantile Fan Chart', ylabel=f'{column}_quantile')

    # Quantile curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    quatile_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Quantile', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return quatile_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='0.25', y2='0.75', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * quatile_chart
    return chart


### Balances and Supplies

In [76]:
system_balances = ['other_issuance_balance', 'reward_issuance_balance']
agent_balances = [
    'farmers_balance',
    'operators_balance',
    'nominators_balance',
    'holders_balance',
]
agent_pool_balances = ['staking_pool_balance']
protocol_treasury_balances = ['fund_balance']
other_balances = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances + agent_balances + agent_pool_balances + protocol_treasury_balances) )
supply_columns = list({c for c in sim_df.columns if 'supply' in c} - {'max_credit_supply', 'issued_supply', 'total_supply'})
balance_columns = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances))

In [77]:
system_balances

['other_issuance_balance', 'reward_issuance_balance']

In [78]:
agent_balances

['farmers_balance',
 'operators_balance',
 'nominators_balance',
 'holders_balance']

In [79]:
agent_pool_balances

['staking_pool_balance']

In [80]:
protocol_treasury_balances

['fund_balance']

In [81]:
other_balances

['burnt_balance']

In [82]:
supply_columns

['circulating_supply',
 'earned_supply',
 'earned_minus_burned_supply',
 'user_supply']

In [83]:
balance_columns

['holders_balance',
 'nominators_balance',
 'farmers_balance',
 'operators_balance',
 'fund_balance',
 'burnt_balance',
 'staking_pool_balance']

### KPIs

In [84]:
sim_df['issuance'] = sim_df['block_reward'] + sim_df['reference_subsidy']

fees = ['compute_fee_volume','storage_fee_volume']

sim_df['fees'] = sim_df['compute_fee_volume'] + sim_df['storage_fee_volume']

In [85]:
# Compute Fees and Storage Fees

# The dynamics of storage fees vs issuance. Who will dominate at the beginning, storage fees or issues rewards? Note that this is a metric.
# Another metrics of interest, general revenue per timestep, farmers, proposers, voters, and data blocks
# revenue = proposer_reward + storage_fees. For data blocks and voters you only have rewards not fees. Farmers is the sum of those three.
# The above topics are what has been discussed and therefor are higher priority than the stocks. 

In [86]:
color_palette = Category20
# columns_to_color = sorted(list(set(sim_df.columns) - {'environmental_label', 'label', 'run', 'blocks_passed', 'days_passed'}))
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}


sim_df.count().to_frame().T.hvplot.bar(y=columns_to_color, color=[column_colors[c] for c in columns_to_color], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

:Bars   [index,Variable]   (value)

In [87]:
balance_columns = ['fees', 'issuance']
balance_columns = supply_columns

In [88]:
box_df = sim_df.set_index(['days_passed', 'label'])[balance_columns]
box_df

,,circulating_supply,earned_supply,earned_minus_burned_supply,user_supply
days_passed,label,,,,
0,constant-single-component,0.000000e+00,0.0,0.0,0.000000e+00
1,constant-single-component,1.369863e+01,0.0,0.0,1.369863e+01
2,constant-single-component,2.736301e+01,0.0,0.0,2.739726e+01
3,constant-single-component,4.120104e+01,0.0,0.0,4.130369e+01
4,constant-single-component,5.521174e+01,0.0,0.0,5.541740e+01
...,...,...,...,...,...
727,hybrid-two-components,1.550348e+08,0.0,0.0,5.140144e+08
728,hybrid-two-components,1.549819e+08,0.0,0.0,5.148093e+08
729,hybrid-two-components,1.549293e+08,0.0,0.0,5.156042e+08


In [89]:
describe_df = box_df.describe().drop('count')
describe_df

,circulating_supply,earned_supply,earned_minus_burned_supply,user_supply
mean,8.710912e+07,0.0,0.0,1.861611e+08
std,8.752173e+07,0.0,0.0,1.951076e+08
min,0.000000e+00,0.0,0.0,0.000000e+00
25%,3.780745e+03,0.0,0.0,6.404602e+03
50%,1.548429e+08,0.0,0.0,2.254190e+08
75%,1.703039e+08,0.0,0.0,3.716143e+08
max,2.250154e+08,0.0,0.0,5.171940e+08


In [90]:
describe_labels_df = box_df.groupby('label').apply(lambda label: label.describe().drop('count'))
describe_labels_df

circulating_supply  earned_supply  \
label                                                               
constant-single-component mean        8.710434e+07            0.0   
                          std         8.755972e+07            0.0   
                          min         0.000000e+00            0.0   
                          25%         2.642554e+03            0.0   
                          50%         1.548348e+08            0.0   
                          75%         1.702928e+08            0.0   
                          max         2.250037e+08            0.0   
hybrid-single-component   mean        8.710887e+07            0.0   
                          std         8.756092e+07            0.0   
                          min         0.000000e+00            0.0   
                          25%         6.332471e+03            0.0   
                          50%         1.548409e+08            0.0   
                          75%         1.702986e+08            0.0   
                          max         2.250090e+08            0.0   
hybrid-two-components     mean        8.711416e+07            0.0   
                          std         8.756425e+07            0.0   
                          min         0.000000e+00            0.0   
                          25%         6.352281e+03            0.0   
                          50%         1.548509e+08            0.0   
                          75%         1.703076e+08            0.0   
                          max         2.250154e+08            0.0   

                                earned_minus_burned_supply   user_supply  
label                                                                     
constant-single-component mean                         0.0  1.861501e+08  
                          std                          0.0  1.951895e+08  
                          min                          0.0  0.000000e+00  
                          25%                          0.0  3.912143e+03  
                          50%                          0.0  2.254086e+08  
                          75%                          0.0  3.715873e+08  
                          max                          0.0  5.171427e+08  
hybrid-single-component   mean                         0.0  1.861612e+08  
                          std                          0.0  1.951955e+08  
                          min                          0.0  0.000000e+00  
                          25%                          0.0  9.402103e+03  
                          50%                          0.0  2.254197e+08  
                          75%                          0.0  3.716041e+08  
                          max                          0.0  5.171650e+08  
hybrid-two-components     mean                         0.0  1.861721e+08  
                          std                          0.0  1.952048e+08  
                          min                          0.0  0.000000e+00  
                          25%                          0.0  9.421913e+03  
                          50%                          0.0  2.254294e+08  
                          75%                          0.0  3.716234e+08  
                          max                          0.0  5.171940e+08

In [91]:
describe_difference_df = pd.DataFrame(describe_labels_df.values - pd.concat([describe_df for i in range(sim_df['label'].nunique())]).values, columns=describe_labels_df.columns, index=describe_labels_df.index)
df = describe_difference_df

def log_scale(val, max_abs_log):
    """ Apply logarithmic scaling to a value. """
    if val == 0:
        return 0
    else:
        return np.sign(val) * np.log(abs(val) + 1) / max_abs_log

def color_scale(val):
    max_abs_val = df.abs().max().max()
    max_abs_log = np.log(max_abs_val + 1)

    scaled_val = log_scale(val, max_abs_log)

    if scaled_val < 0:
        intensity = int(255 * (1 + scaled_val))  # More negative, more red
        return f'background-color: rgb(255, {intensity}, {intensity})'
    elif scaled_val > 0:
        intensity = int(255 * (1 - scaled_val))  # More positive, more green
        return f'background-color: rgb({intensity}, 255, {intensity})'
    else:
        return 'background-color: rgb(255, 255, 255)'

header_styles = [{
    'selector': f'th.col_heading.level0.col{i}',
    'props': [('background-color', column_colors.get(col))]
} for i, col in enumerate(df.columns)]

df.columns.name = 'balance'

describe_difference_df_styled = df.style.map(color_scale).set_table_styles(header_styles)
describe_difference_df_styled

In [96]:
# create violin charts side by side for month 1 and month 13 for each label
def compare_violin_charts(df):
    _df = df.copy()
    df = df.reset_index()
    df['month'] = df['days_passed'].apply(lambda x: int(x/30)+1)
    df = df[df['month'].isin([1,13])]
    df = df.melt(id_vars=['month', 'label'], value_vars=df.columns[2:-1])

    df = [df[df['month']==1].drop('month', axis=1), df[df['month']==13].drop('month', axis=1)]

    violin_list_1 = []
    for name, label in df[0].groupby('label'):
        violin_list_1.append(label.hvplot.violin(y='value', by='variable', c='variable', legend='top_left', width=600, height=500, title=f'SSC Balances {name} Month 1', cmap=column_colors, ylim=(0,_df.max().max()*0.75)))
    violin_list_2 = []
    for name, label in df[1].groupby('label'):
        violin_list_2.append(label.hvplot.violin(y='value', by='variable', c='variable', legend='top_left', width=600, height=500, title=f'SSC Balances {name} Month 13', cmap=column_colors, ylim=(0,_df.max().max()*0.75)))

    v = []
    for v1, v2 in zip(violin_list_1, violin_list_2):
        v.append(v1)
        v.append(v2)
    
    layout = hv.Layout(v).cols(2)
    return layout

layout = compare_violin_charts(box_df)
layout


:Layout
   .Violin.I   :Violin   [variable]   (value)
   .Violin.II  :Violin   [variable]   (value)
   .Violin.III :Violin   [variable]   (value)
   .Violin.IV  :Violin   [variable]   (value)
   .Violin.V   :Violin   [variable]   (value)
   .Violin.VI  :Violin   [variable]   (value)

In [92]:
box_df_melted = box_df.reset_index().drop('days_passed',axis=1).melt(id_vars=['label'])

violin_list = [label.hvplot.violin(y='value', by='variable', c='variable', legend='top_left', width=1200, height=500, title=f'SSC Balances {name}', cmap=column_colors, ylim=(0,box_df.max().max()*0.75)) for name, label in box_df_melted.groupby('label')]

# Combine plots into a single column layout
layout = hv.Layout(violin_list).cols(1)

layout

:Layout
   .Violin.I   :Violin   [variable]   (value)
   .Violin.II  :Violin   [variable]   (value)
   .Violin.III :Violin   [variable]   (value)

In [93]:
line_list = [hv.Overlay([fan_chart_quantile(label, column) for column in label.columns if column not in ['label', 'days_passed']]).opts(title=f'SSC Balances {name}', legend_opts={'location':'top_left'}) for name, label in box_df.reset_index().groupby('label')]
layout = hv.Layout(line_list).cols(1)
layout

:Layout
   .Overlay.I   :Overlay
      .Area.I                                    :Area   [days_passed]   (0.25,0.75)
      .Curve.Circulating_Supply_Quantile         :Curve   [days_passed]   (median)
      .Area.II                                   :Area   [days_passed]   (0.25,0.75)
      .Curve.Earned_Supply_Quantile              :Curve   [days_passed]   (median)
      .Area.III                                  :Area   [days_passed]   (0.25,0.75)
      .Curve.Earned_Minus_Burned_Supply_Quantile :Curve   [days_passed]   (median)
      .Area.IV                                   :Area   [days_passed]   (0.25,0.75)
      .Curve.User_Supply_Quantile                :Curve   [days_passed]   (median)
   .Overlay.II  :Overlay
      .Area.I                                    :Area   [days_passed]   (0.25,0.75)
      .Curve.Circulating_Supply_Quantile         :Curve   [days_passed]   (median)
      .Area.II                                   :Area   [days_passed]   (0.25,0.75)
      .Curve.Earned_Supply_Quantile              :Curve   [days_passed]   (median)
      .Area.III                                  :Area   [days_passed]   (0.25,0.75)
      .Curve.Earned_Minus_Burned_Supply_Quantile :Curve   [days_passed]   (median)
      .Area.IV                                   :Area   [days_passed]   (0.25,0.75)
      .Curve.User_Supply_Quantile                :Curve   [days_passed]   (median)
   .Overlay.III :Overlay
      .Area.I                                    :Area   [days_passed]   (0.25,0.75)
      .Curve.Circulating_Supply_Quantile         :Curve   [days_passed]   (median)
      .Area.II                                   :Area   [days_passed]   (0.25,0.75)
      .Curve.Earned_Supply_Quantile              :Curve   [days_passed]   (median)
      .Area.III                                  :Area   [days_passed]   (0.25,0.75)
      .Curve.Earned_Minus_Burned_Supply_Quantile :Curve   [days_passed]   (median)
      .Area.IV                                   :Area   [days_passed]   (0.25,0.75)
      .Curve.User_Supply_Quantile                :Curve   [days_passed]   (median)

In [94]:
violin_list = [variable.hvplot.violin(y='value', by='label', color=column_colors[name], width=1200, height=500, title=f'SSC Balances {name}', ylim=(0,variable.max()['value'].max())) for name, variable in box_df_melted.groupby('variable')]

layout = hv.Layout(violin_list).cols(1).opts(shared_axes=False)

layout

:Layout
   .Violin.I   :Violin   [label]   (value)
   .Violin.II  :Violin   [label]   (value)
   .Violin.III :Violin   [label]   (value)
   .Violin.IV  :Violin   [label]   (value)

In [95]:
line_list = [variable.hvplot.line(x='days_passed', by='label', y='value', title=name, legend='top_left', line_width=3).opts(legend_opts={'background_fill_color': column_colors[name], 'background_fill_alpha': 0.2}) for name, variable in box_df.reset_index().melt(id_vars=['label', 'days_passed']).groupby('variable')]

layout = hv.Layout(line_list).cols(2).opts(shared_axes=False)
layout

:Layout
   .NdOverlay.I   :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.II  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.III :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.IV  :NdOverlay   [label]
      :Curve   [days_passed]   (value)